In [1]:
from gensim.models.doc2vec import Doc2Vec
from scipy.spatial.distance import cosine, cdist

In [2]:
d2v = Doc2Vec.load("educaloi_cappel_d2v.model")

In [3]:
import sqlite3
cxn = sqlite3.connect("/home/mok/Sync/Projets/Juridico/educaloi.sqlite")
cs = cxn.cursor()

# Insérer les articles Éducaloi, automatiquement tagués

In [4]:
tags_ls = [o.nom for o in Categorie.objects.all()]

In [5]:
tag_names = [ "TAG_"+i for i in tags_ls]
tag_vecs = [d2v.docvecs[i] for i in tag_names]

In [6]:
def get_el_tags(elid, topn=30):
    d = cdist([d2v.docvecs["EL_%d"%elid]],tag_vecs)
    return [ (i, r[0], r[1][4:]) for i, r in enumerate(sorted(zip(d[0], tag_names))[:topn])]

In [7]:
cs.execute("select artid, nom, description, cat, url from articles")
for artid, nom, desc, elcat, url in cs.fetchall():
    elart = Educaloi.objects.create(
        nom=nom,
        artid=artid,
        description=desc,
        url=url,
        categorie_educaloi=elcat
    )
    elart.save()
    for rang, dist, categ in get_el_tags(artid):
        elartcat = EducaloiCategsPonderees.objects.create(
            article=elart,
            rang=rang,
            distance=dist,
            categorie=Categorie.objects.get(nom=categ)
        )
        elartcat.save()

In [12]:
import re
import numpy as np
is_tag = re.compile("^TAG_")
is_el = re.compile("^EL_")
is_eyb = re.compile("^EYB_")

In [28]:
np.nonzero(np.fromiter((is_tag.match(i) !=None for i in d2v.docvecs.index2entity), dtype=bool))[0]

array([   0,    1,    2,    3,    4,    5,    6,    7,    9,   10,   12,
         13,   14,   15,   16,   17,   19,   20,   21,   22,   23,   24,
         25,   26,   27,   29,   30,   31,   32,   33,   34,   35,   36,
         37,   39,   40,   41,   42,   43,   45,   46,   47,   48,   49,
         50,   52,   53,   54,   56,   57,   58,   59,   61,   63,   64,
         66,   67,   68,   69,   70,   71,   72,   73,   74,   75,   76,
         78,   79,   81,   83,   84,   85,   86,   87,   89,   90,   91,
         92,   94,   95,   97,   98,   99,  100,  101,  102,  104,  106,
        108,  109,  110,  112,  113,  114,  115,  117,  118,  119,  121,
        123,  124,  125,  126,  128,  129,  130,  131,  133,  134,  136,
        137,  138,  140,  141,  142,  143,  144,  145,  147,  149,  150,
        151,  152,  154,  156,  157,  158,  160,  162,  163,  164,  166,
        167,  169,  170,  171,  172,  175,  176,  177,  178,  179,  180,
        182,  183,  185,  186,  187,  188,  189,  1

In [32]:
d2v.docvecs.vectors_docs[np.array((1,2))].shape

(2, 100)

In [33]:
cdist?